In [1]:
import numpy as np
import cv2
#import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from PIL import Image
#from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing import image
import collections
import cv2
#from google.colab.patches import cv2_imshow
import glob
from skimage.transform import resize
#import tensorflow as tf
#from sklearn.preprocessing import LabelEncoder
#import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
import torch
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax , Softmax , functional
from torch import flatten
from torch.optim import Adam
import torchvision
from torch.utils.data import DataLoader



Extracting the train and test datasets

In [ ]:
TrainFile = "data.zip"

with ZipFile(TrainFile, 'r') as zip:
	zip.printdir()
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

Haar Cascade function: takes an image finds the faces and returns them cropped 


In [3]:
def haarCascade(img):
 face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
 eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
 gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 faces = face_cascade.detectMultiScale(gray, 1.3, 5,minSize=(30, 30))
 
 for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    roi_color = img[y:y+h, x:x+w]
    return roi_color


In [ ]:
faces = "faces.zip"

with ZipFile(faces, 'r') as zip:
	zip.printdir()
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

In [ ]:
path='/content/data/menna/9.jpeg'
img =  haarCascade(cv2.imread(path))
img = Image.fromarray(img)

This Loop iterates over the dataset and returns a file called faces that has all the saved images after applying the haar cascade algorithm.

In [6]:
path1 = "data/*/*"
target_face_size=(388,388)

path='faces/'
i=0
for file in glob.glob(path1):
  if('jumana' in file):
    img =  haarCascade(cv2.imread(file))
    img = Image.fromarray(img)
    img = img.resize(target_face_size)
    cv2.imwrite(str(path)+'jumana/'+str(i) + '.jpg',np.array(img))
    i=i+1
  # elif('farida' in file):
  #   img =  haarCascade(cv2.imread(file))
  #   img = Image.fromarray(img)
  #   img = img.resize(target_face_size)
  #   cv2.imwrite(str(path)+'farida/'+str(i) + '.jpg',np.array(img))  
  #   i=i+1
  elif('kroush'in file):
    img =  haarCascade(cv2.imread(file))
    img = Image.fromarray(img)
    img = img.resize(target_face_size)
    cv2.imwrite(str(path)+'kroush/'+str(i) + '.jpg',np.array(img))
    i=i+1
  # elif('menna'in file):
  #   img =  haarCascade(cv2.imread(file))
  #   img = Image.fromarray(img)
  #   img = img.resize(target_face_size)
  #   cv2.imwrite(str(path)+'menna/'+str(i) + '.jpg',np.array(img))
  #   i=i+1

trans_resize is a way by pytorch to make sure all images are resized. Then we change the dataset to tensors. trainDataLoader takes in the dataset, shuffles it and makes the model take a batch size of one in every time

In [7]:
trans_resize=torchvision.transforms.Resize((388,388))
trainTransforms = torchvision.transforms.Compose([ torchvision.transforms.ToTensor(), trans_resize ])


dataset= torchvision.datasets.ImageFolder('faces',transform=trainTransforms)

trainDataLoader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
 )

**CNN**

We use GPU to make matrix calculations faster

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

This should be the architecture of the Convolutional neural network where we apply a layer of convolution, then we apply relu and then we max pool. after that we add two linear functions then finally we apply the sigmoid function.

In [9]:
class FaceNet(Module):
  def __init__(self):
    # call the parent constructor
    super(FaceNet, self).__init__()
    # initialize first set of CONV => RELU => POOL layers
    self.conv1 = Conv2d(in_channels=3, out_channels=64,
      kernel_size=(3,3))
    self.relu1 = ReLU()
    self.maxpool1 = MaxPool2d(kernel_size=(4, 4), stride=(2, 2))
    # initialize second set of CONV => RELU => POOL layers
   # self.conv2 = Conv2d(in_channels=32, out_channels=64,
     # kernel_size=(3, 3))
    #self.relu2 = ReLU()
    #self.maxpool2 = MaxPool2d(kernel_size=(5, 5), stride=(2, 2))
    
    #self.conv3 = Conv2d(in_channels=64, out_channels=128,
     # kernel_size=(3,3))
    #self.relu3 = ReLU()
    #self.maxpool3 = MaxPool2d(kernel_size=(4, 4), stride=(2, 2))
    # initialize first (and only) set of FC => RELU layers
    self.fc1 = Linear(in_features=2359296, out_features=64)
    self.relu3 = ReLU()
    
    #self.fc2 = Linear(in_features=512, out_features=64)
    #self.relu4 = ReLU()
    # initialize our softmax classifier
    self.fc3 = Linear(in_features=64, out_features=2)
    self.logSoftmax = torch.nn.Sigmoid()

  def forward(self, x):
    # pass the input through our first set of CONV => RELU =>
    # POOL layers
    x = self.conv1(x)
    x = self.relu1(x)
    x = self.maxpool1(x)
    # pass the output from the previous layer through the second
    # set of CONV => RELU => POOL layers
    #x = self.conv2(x)
   # x = self.relu2(x)
    #x = self.maxpool2(x)
    
   # x = self.conv3(x)
   # x = self.relu3(x)
    #x = self.maxpool3(x)
    # flatten the output from the previous layer and pass it
    # through our only set of FC => RELU layers
    x = flatten(x, 1)
    x = self.fc1(x)
    x = self.relu3(x)
    
    #x = self.fc2(x)
    #x = self.relu4(x)
    
    # pass the output to our softmax classifier to get our output
    # predictions
    x = self.fc3(x)
    output = self.logSoftmax(x)
    # return the output predictions
    return output

model = FaceNet().to(device)

we used adam func with learning rate -8 so that the model doesnt overfit.
And BCE as the loss function

In [10]:
opt = Adam(model.parameters(), lr=1e-8)
lossFn = torch.nn.BCEWithLogitsLoss()


In this loop, we calculate the total train loss and the total validation loss

In [ ]:
for e in range(0, 40):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
	
		y=functional.one_hot(y,num_classes = 2)
		print('TRUTH',y[0])

		# perform a forward pass and calculate the training loss
		pred = model(x)
		print('PREDICTION',pred[0])
		loss = lossFn(pred.type(torch.FloatTensor), y.type(torch.FloatTensor))
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		print("LOSS",loss.item())
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y.argmax(1) ).type(torch.float).sum().item()
        
        
        
        

In [12]:
print("Accuracy",trainCorrect/17*100)

Accuracy 100.0


In [13]:
totalTrainLoss

tensor(11.8366, grad_fn=<AddBackward0>)

In [14]:
import matplotlib.pyplot as plt

In [ ]:


for (x, y) in trainDataLoader:
    # send the input to the device
    (x, y) = (x.to(device), y.to(device))

    

    # perform a forward pass and calculate the training loss
    pred = model(x)
    
    img = x[0].to(device,dtype=torch.float).permute(1, 2, 0).detach().cpu().numpy()
    
    plt.imshow(img)
    plt.show()
    y=functional.one_hot(y,num_classes = 2)
    correct = pred.argmax() == y.argmax()
    print(correct,pred,y)



